## 쿼리 연산자

점 표기법 : bson내부의 object에 접근하기 위한 방법

### 비교연산자
$eq (equals) 주어진 값과 일치하는 값

$gt (greater than) 주어진 값보다 큰 값

$gte (greather than or equals) 주어진 값보다 크거나 같은 값

$lt (less than) 주어진 값보다 작은 값

$lte (less than or equals) 주어진 값보다 작거나 같은 값

$ne (not equal) 주어진 값과 일치하지 않는 값

$in 주어진 배열 안에 속하는 값

$nin 주어진 배열 안에 속하지 않는 값


### 논리 연산자
$or 주어진 조건 중 하나라도 true 일 때 true

$and 주어진 모든 조건이 true 일 때 true >> 복합적일 때 자주 사용.

$nor 주어진 조건 중 하나라도 false 일 때 true

$not 주어진 조건이 false 일 때 true


### 문자열 연산자
$mod 필드에 modulo operation을 통해 특정 결과가 나온 Document를 선택한다.

$regex 특정 정규 표현식과 맞는 Document를 선택한다

$text 문자열 검색의 기능을 수행한다

$where 자바스크립트로 알맞은 Document를 선택한다

1.정규표현식 연산자(regex)
{ <field>: { "$regex": 'pattern', "$options": '<options>' } }

options / 설명

i - 대소문자 무시

m - 정규식에서 anchor(^) 를 사용할 때 값에 \n 이 있다면 무력화

x - 정규식 안에 있는 whitespace를 모두 무시

s dot - (.) 사용 할 때 \n 을 포함해서 매치

2.Text 연산자

{"$text": { "$search": <string>, "$language": <string>, "$caseSensitive": <boolean>, "$diacriticSensitive": <boolean> }}

options / 설명
    
$search - 검색할 내용

$language - 선택적.검색하는 언어
    
$caseSensitive - 선택적.False일 경우 대소문자 무시. False가 기본값

$diacriticSensitive - 선택적. ğ와 g 같이 diacritical mark를 구분할지 선택. False가 기본값

문자열 인덱스 설정
    
collection.create_index([('field', pymongo.TEXT)], default_language='english')
컬렉션당 하나만 만들 수 있는 문자열 인덱스에서만 작동함
안타깝게도 한국어는 문자열 인덱스로 지원하지 않음

    
### 배열 연산자
    
$all 순서와 상관없이 배열 안의 요소가 모두 포함되면 선택한다

$elemMatch $elemMatch 조건과 맞는 배열 속 요소를 가진 Document를 선택한다.

$size 해당 배열의 크기가 같은 Document를 선택한다.

## 실습1. 점 표기법

* 지시사항

1. name의 last 값이 Cate인 하나의 사람 도큐먼트에서 skills 필드를 제거하세요.

2. 수정한 도큐먼트를 pprint()로 출력하세요.

In [2]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("profile")
col = db.get_collection("people")

# name의 last 값이 "Cate"인 하나의 사람 도큐먼트 조건을 작성하세요.
query = {"name.last" : "Cate"}

# 해당 도큐먼트를 수정하세요.
result = col.update_many(query, {"$unset": {"skills": False}})

# 수정한 도큐먼트를 조회하세요.
cursor = col.find(query)

# 앞선 결과를 출력하세요.
for people in cursor:
    pprint(people)

## 실습2-1. 비교 연산자

1. date_received의 값은 예시와 같은 형식의 문자열로 저장되어 있습니다.(예시: "2012-04-01")

2. date_received 값이 2015년도 혹은 2017년도가 아닌 책들을 모두 삭제하세요.

In [8]:
#nin
import pymongo
from pprint import pprint
import re

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# date_received 값이 2015년도 혹은 2017년도가 아닌 조건을 작성하세요.
query = {"date_received" : {"$nin" : [re.compile("^2015"), re.compile("^2017")]}}

# 조건에 맞는 도큐먼트를 삭제하세요.
delete = col.delete_many(query)

print('____자동출력: 삭제 후 남은 books의 모든 도큐먼트____')
for x in col.find():
    print(x)

____자동출력: 삭제 후 남은 books의 모든 도큐먼트____
{'_id': ObjectId('615be8dab02c2e24d41c01dd'), 'title': 'Harry Potter and the Deathly Hallows', 'author': 'Joanne Kathleen Rowling', 'publisher': 'Bloomsbury Publishing', 'date_received': '2017-07-21'}


## 실습2-2. 비교연산자

1. date_received의 값은 다음과 같은 형식의 문자열로 저장되어 있습니다.(예시: "2012-04-01")

2. date_received값이 2014년도부터 2017년도인 책들을 조회하세요.

3. 그 책들의 '_id'와 'title' 필드만 도큐먼트 형식으로 for 문을 사용하여 pprint()로 출력해보세요.

In [9]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# date_received 값이 2014년도부터 2017년도인 책들의 title 필드만 조회하세요.
query = {"date_received" : {"$gte": "2014-01-01", "$lte": "2017-12-31"}}
projection = {"_id" : True, "title" : True}
cursor = col.find(query, projection)

# books Collection에 들어있는 책들을 출력하세요.

for book in cursor:
    pprint(book)

{'_id': ObjectId('615be8dab02c2e24d41c01dd'),
 'title': 'Harry Potter and the Deathly Hallows'}


## 실습3. 논리 연산자

1. date_received의 값은 다음과 같은 형식의 문자열로 저장되어 있습니다. (예시: "2012-04-01")

2. "date_received"가 2014년도 또는 2019년도인 책 도큐먼트를 찾으세요.

3. 찾은 도큐먼트를 for 문을 사용해 pprint()로 출력하세요.

In [10]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# date_received가 2014년도인 책을 찾는 조건을 작성하세요.
query1 = {"date_received" : {"$regex" : "^2014"}}
# date_received가 2019년도인 책을 찾는 조건을 작성하세요.
query2 = {"date_received" : {"$regex" : "^2019"}}
# query1 또는 query2를 충족하는 책을 찾아주세요
query = {"$or" : [query1, query2]}
cursor = col.find(query)

# books Collection에 들어있는 책들을 출력하세요.
for book in cursor:
    pprint(book)

## 실습4. 복합 논리 연산자

1. date_received의 값은 다음과 같은 형식의 문자열로 저장되어 있습니다. (예시: "2012-04-01")

2.다음 조건을 모두 만족시키는 책 도큐먼트를 찾으세요

"author"가 "Antoine de Saint-Exupery" 또는 "Ernest Miller Hemingway"인 책

"date_received"가 2014년도 또는 2019년도인 책

3. 찾은 도큐먼트를 for 문을 사용해 pprint()로 출력하세요

In [11]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# author가 "Antoine de Saint-Exupery" 또는 "Ernest Miller Hemingway"인 책을 찾는 조건을 작성하세요.
query1 = {"$or" : [{"author" : "Antoine de Saint-Exupery"}, {"author" : "Ernest Miller Hemingway"}]}
# "date_received"가 2014년도 또는 2019년도인 책을 찾는 조건을 작성하세요.
query2 = {"$or" : [{"date_received" : {"$regex" : "^2014"}}, {"date_received" : {"$regex" : "^2019"}}]}
# query1과 query2를 모두 충족하는 책을 찾아주세요.
query = {"$and" : [query1, query2]}
cursor = col.find(query)

# books Collection에 들어있는 책들을 출력하세요.
for book in cursor:
    pprint(book)

## 실습5. 문자열 연산자

1. title 필드에 harry라는 문자열이 있는 책을 texttexttext 연산자를 활용해 찾으세요.

In [12]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("library")
col = db.get_collection("books")

# 텍스트 인덱스를 title 필드에 생성하는 코드입니다.
col.create_index([('title', pymongo.TEXT)], default_language='english')

# 제목에 harry가 있는 책을 $search 연산자를 활용해 조회하세요.
query = {"$text" : {"$search" : "harry"}}
cursor = col.find(query)

# 결과를 출력합니다.
for book in cursor:
    pprint(book)

{'_id': ObjectId('615be8dab02c2e24d41c01dd'),
 'author': 'Joanne Kathleen Rowling',
 'date_received': '2017-07-21',
 'publisher': 'Bloomsbury Publishing',
 'title': 'Harry Potter and the Deathly Hallows'}


## 실습6-1 배열연산자

1. Python 언어의 level 값이 3인 skills를 가지고 있는 도큐먼트를 모두 출력하세요.

In [13]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("profile")
col = db.get_collection("people")

# Python 언어의 level 값이 3인 skills를 가지고 있는 사람 도큐먼트를 조회하세요.
query = {"skills" : {"$elemMatch" : {"name" : "Python", "level" : 3}}}
cursor = col.find(query)

# 결과를 출력합니다.
for people in cursor:
    pprint(people)

## 실습6-2 배열연산자

* 지시사항

1. Ruby와 Python 프로그래밍 skills를 다룰 수 있는 사람 도큐먼트를 모두 출력하세요.

2. 프로그래밍 skills를 4개 가진 사람 도큐먼트를 모두 출력하세요.

In [14]:
import pymongo
from pprint import pprint

# 데이터베이스와 Collection을 생성하는 코드입니다. 수정하지 마세요!
connection = pymongo.MongoClient("mongodb://localhost:27017/")
db = connection.get_database("profile")
col = db.get_collection("people")

# Ruby와 Python 프로그래밍 skills를 다룰 수 있는 사람 도큐먼트를 조회
query = {"skills.name" : {"$all" : ["Ruby", "Python"] }}
cursor = col.find(query)

# 결과를 출력합니다.
for people in cursor:
    pprint(people)

# # 프로그래밍 skills을 4개 가진 사람 도큐먼트들을 검색
query = {"skills" : {"$size" : 4}}
cursor = col.find(query)

# 결과를 출력합니다.
for people in cursor:
    pprint(people)